In [8]:
# city scapes
#
import os
import imageio
from pathlib import Path

In [7]:
from PIL import Image


In [ ]:
## remove non essential files

In [2]:
directory= "../cityscapes_pretraiement/gtFine/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    if not f.endswith("_labelTrainIds.png"):
        os.remove(filename)

In [3]:
directory= "../cityscapes_pretraiement/gtFine/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    if not f.endswith("_labelTrainIds.png"):
        os.remove(filename)

In [ ]:
## resize

In [11]:
directory= "../cityscapes_pretraiement/gtFine/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [12]:
directory= "../cityscapes_pretraiement/gtFine/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [15]:
directory= "../cityscapes_pretraiement/leftImg8bit/train/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [16]:
directory= "../cityscapes_pretraiement/leftImg8bit/val/"
for f in os.listdir(directory):
    filename = directory+""+f
    img = Image.open(filename)
    img = img.resize((128,128), Image.ANTIALIAS)
    img.save(directory+""+f)

In [17]:
##### renaming files

In [27]:
import os

directory= "cityscapes_data/annotations/validation/"
FList = os.listdir(directory)
FListC = FList[1:]
m = 0
for i in FListC:
    print(i)
    fileExtension = os.path.splitext(i)[1]
    os.rename(directory+""+i,directory+""+str(m)+fileExtension)
    m = m+1


frankfurt_000000_000576_leftImg8bit.png
frankfurt_000000_001016_leftImg8bit.png
frankfurt_000000_001236_leftImg8bit.png
frankfurt_000000_001751_leftImg8bit.png
frankfurt_000000_002196_leftImg8bit.png
frankfurt_000000_002963_leftImg8bit.png
frankfurt_000000_003025_leftImg8bit.png
frankfurt_000000_003357_leftImg8bit.png
frankfurt_000000_003920_leftImg8bit.png
frankfurt_000000_004617_leftImg8bit.png
frankfurt_000000_005543_leftImg8bit.png
frankfurt_000000_005898_leftImg8bit.png
frankfurt_000000_006589_leftImg8bit.png
frankfurt_000000_007365_leftImg8bit.png
frankfurt_000000_008206_leftImg8bit.png
frankfurt_000000_008451_leftImg8bit.png
frankfurt_000000_009291_leftImg8bit.png
frankfurt_000000_009561_leftImg8bit.png
frankfurt_000000_009688_leftImg8bit.png
frankfurt_000000_009969_leftImg8bit.png
frankfurt_000000_010351_leftImg8bit.png
frankfurt_000000_010763_leftImg8bit.png
frankfurt_000000_011007_leftImg8bit.png
frankfurt_000000_011074_leftImg8bit.png
frankfurt_000000_011461_leftImg8bit.png


In [29]:
import os

directory= "cityscapes_data/annotations/training/"
FList = os.listdir(directory)
FListC = FList[1:]
m = 0
for i in FListC:
    print(i)
    fileExtension = os.path.splitext(i)[1]
    os.rename(directory+""+i,directory+""+str(m)+fileExtension)
    m = m+1


9_gtFine_labelTrainIds.png
tubingen_000066_000019_gtFine_labelTrainIds.png
tubingen_000067_000019_gtFine_labelTrainIds.png
tubingen_000068_000019_gtFine_labelTrainIds.png
tubingen_000069_000019_gtFine_labelTrainIds.png
tubingen_000070_000019_gtFine_labelTrainIds.png
tubingen_000071_000019_gtFine_labelTrainIds.png
tubingen_000072_000019_gtFine_labelTrainIds.png
tubingen_000073_000019_gtFine_labelTrainIds.png
tubingen_000074_000019_gtFine_labelTrainIds.png
tubingen_000075_000019_gtFine_labelTrainIds.png
tubingen_000076_000019_gtFine_labelTrainIds.png
tubingen_000077_000019_gtFine_labelTrainIds.png
tubingen_000078_000019_gtFine_labelTrainIds.png
tubingen_000079_000019_gtFine_labelTrainIds.png
tubingen_000080_000019_gtFine_labelTrainIds.png
tubingen_000081_000019_gtFine_labelTrainIds.png
tubingen_000082_000019_gtFine_labelTrainIds.png
tubingen_000083_000019_gtFine_labelTrainIds.png
tubingen_000084_000019_gtFine_labelTrainIds.png
tubingen_000085_000019_gtFine_labelTrainIds.png
tubingen_0000